# Imports and Colab Mount

In [1]:
import datetime
import seaborn as sn
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as stats
import glob
from math import sqrt

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, LSTM, LeakyReLU, Dropout, GRU, SimpleRNN, Input, LSTM, Dense, Bidirectional, Concatenate, Reshape, Lambda, Bidirectional
from keras.models import Model, Sequential
from keras import backend as K
from tensorflow.keras import layers
from keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
import seaborn as sns

from numpy.random import seed
#from tensorflow import set_random_seed

%matplotlib inline

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Load Data

In [ ]:
hourly = pd.read_excel("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/BASENATALhorario.xlsx", index_col=0, parse_dates=[["Data", "HORA (UTC)"]])
hourly = hourly[ ['Velocidade'] + [ col for col in hourly.columns if col != 'Velocidade' ] ]
quarto = int(hourly["Velocidade"].shape[0]/4)

#resid_train = pd.read_csv("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/natalhora_resid_train.csv").drop("Unnamed: 0", axis=1)
#resid_test = pd.read_csv("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/natalhora_resid_test2.csv").drop("Unnamed: 0", axis=1)

sarima_pred = pd.read_csv("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/natalhora_prediction_bad.csv").drop("Unnamed: 0", axis=1)
sarima_pred2 = pd.read_csv("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/natalhora_prediction_good.csv").drop("Unnamed: 0", axis=1)

#hourly["Velocidade"] = pd.concat([resid_train, resid_test]).reset_index()["0"].values
resid_series = pd.read_csv("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/natalhora_resid_series.csv").drop("Unnamed: 0", axis=1)

train_h = hourly.iloc[:quarto*2]
valid_h = hourly.iloc[quarto*2:quarto*3]
test_h = hourly.iloc[quarto*3:]

In [ ]:
hourly['Velocidade'].to_csv("/content/drive/MyDrive/Hybrid Transformer/Datasets/Vento/natalvelocidade.csv")

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
def make_data(data, timestep, resid_check=False):
  values = data.values
  values = values.astype('float32')
  #scaler = MinMaxScaler(feature_range=(0, 1))
  #scaled = scaler.fit_transform(values)
  
  timestep = timestep
  n_features = data.shape[1]
  n_obs = timestep * n_features
  reframed = series_to_supervised(values, timestep, 1)
  reframed = reframed.iloc[: , :1-n_features]

  values = reframed.values
  indice1 = train_h.shape[0]
  indice2 = valid_h.shape[0]

  train = values[:indice1, :]
  valid = values[indice1:indice1+indice2, :]
  test = values[indice1+indice2:, :]

  train_X, train_y = train[:, :-1], train[:, -1]
  valid_X, valid_y = valid[:, :-1], valid[:, -1]
  test_X, test_y = test[:, :-1], test[:, -1]

  if (resid_check==True):
    train_y = resid_train.values[:quarto*2]
    valid_y = resid_train.values[quarto*2:]

  scaler = MinMaxScaler(feature_range=(0, 1)).fit(train_X)
  train_X = scaler.transform(train_X)
  valid_X = scaler.transform(valid_X)
  test_X = scaler.transform(test_X)

  scaler_y = MinMaxScaler(feature_range=(0, 1)).fit(train_y.reshape(-1,1))
  train_y = scaler_y.transform(train_y.reshape(-1,1))
  valid_y = scaler_y.transform(valid_y.reshape(-1,1))
  test_y = scaler_y.transform(test_y.reshape(-1,1))

  train_X = train_X.reshape((train_X.shape[0], timestep, n_features))
  valid_X = valid_X.reshape((valid_X.shape[0], timestep, n_features))
  test_X = test_X.reshape((test_X.shape[0], timestep, n_features))
  return train_X, train_y, valid_X, valid_y, test_X, test_y

In [ ]:
def make_data2(data, timestep, resid_check=False):
  values = data.values
  values = values.astype('float32')
  #scaler = MinMaxScaler(feature_range=(0, 1))
  #scaled = scaler.fit_transform(values)
  
  timestep = timestep
  n_features = data.shape[1]
  n_obs = timestep * n_features
  reframed = series_to_supervised(values, timestep, 1)
  reframed = reframed.iloc[: , :1-n_features]

  values = reframed.values
  indice1 = train_h.shape[0]
  indice2 = valid_h.shape[0]

  train = values[:indice1, :]
  valid = values[indice1:indice1+indice2, :]
  test = values[indice1+indice2:, :]

  train_X, train_y = train[:, :-1], train[:, -1]
  valid_X, valid_y = valid[:, :-1], valid[:, -1]
  test_X, test_y = test[:, :-1], test[:, -1]

  if (resid_check==True):
    train_y = resid_train.values[:quarto*2]
    valid_y = resid_train.values[quarto*2:]

  scaler = MinMaxScaler(feature_range=(0, 1)).fit(train_X)
  train_X = scaler.transform(train_X)
  valid_X = scaler.transform(valid_X)
  test_X = scaler.transform(test_X)

  scaler_y = MinMaxScaler(feature_range=(0, 1)).fit(train_y.reshape(-1,1))
  train_y = scaler_y.transform(train_y.reshape(-1,1))
  valid_y = scaler_y.transform(valid_y.reshape(-1,1))
  test_y = scaler_y.transform(test_y.reshape(-1,1))

  train_X = train_X.reshape((train_X.shape[0], timestep, n_features))
  valid_X = valid_X.reshape((valid_X.shape[0], timestep, n_features))
  test_X = test_X.reshape((test_X.shape[0], timestep, n_features))
  return train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y

In [ ]:
def make_data3(df, timestep, resid_check=False, resid=False, univariate=False):

  data = df.copy()

  if resid==True:
    data['Residual'] = resid_series.values

  if univariate==True:
    values = data["Velocidade"].values
    values = values.astype('float32')
    values = values.reshape(-1, 1)

    timestep = timestep
    n_features = 1
    n_obs = timestep * n_features
    reframed = series_to_supervised(values, timestep, 1)

  else:
    values = data.values
    values = values.astype('float32')
    #scaler = MinMaxScaler(feature_range=(0, 1))
    #scaled = scaler.fit_transform(values)
    
    timestep = timestep
    n_features = data.shape[1]
    n_obs = timestep * n_features
    reframed = series_to_supervised(values, timestep, 1)
    reframed = reframed.iloc[: , :1-n_features]

  values = reframed.values
  indice1 = train_h.shape[0]
  indice2 = valid_h.shape[0]

  train = values[:indice1, :]
  valid = values[indice1:indice1+indice2, :]
  test = values[indice1+indice2:, :]

  train_X, train_y = train[:, :-1], train[:, -1]
  valid_X, valid_y = valid[:, :-1], valid[:, -1]
  test_X, test_y = test[:, :-1], test[:, -1]

  if (resid_check==True):
    train_y = resid_series.values[timestep:(quarto*2)+timestep]
    valid_y = resid_series.values[(quarto*2)+timestep:(quarto*3)+timestep]
    #test_y = resid_series.values[(quarto*3)+timestep:]
  #if (resid==True):
  #  test_y = df['Velocidade'].values[(quarto*3)+timestep:]

  scaler = MinMaxScaler(feature_range=(0, 1)).fit(train_X)
  train_X = scaler.transform(train_X)
  valid_X = scaler.transform(valid_X)
  test_X = scaler.transform(test_X)

  scaler_y = MinMaxScaler(feature_range=(0, 1)).fit(train_y.reshape(-1,1))
  train_y = scaler_y.transform(train_y.reshape(-1,1))
  valid_y = scaler_y.transform(valid_y.reshape(-1,1))
  test_y = scaler_y.transform(test_y.reshape(-1,1))

  train_X = train_X.reshape((train_X.shape[0], timestep, n_features))
  valid_X = valid_X.reshape((valid_X.shape[0], timestep, n_features))
  test_X = test_X.reshape((test_X.shape[0], timestep, n_features))
  return train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y

# WanDB

In [ ]:
!pip install wandb -qq
import wandb
from wandb.keras import WandbCallback
!wandb login

     |████████████████████████████████| 1.8 MB 5.1 MB/s 
     |████████████████████████████████| 145 kB 52.4 MB/s 
     |████████████████████████████████| 181 kB 54.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: ERROR No API key specified.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Train

# RNN

In [ ]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'name': 'RNN',
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'best_val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'batch_size': {
            'values': [256, 128, 64, 32]
        },
        'dropout': {
            'values': [0, 0.1, 0.3]
        },
        'units': {
            'values': [32, 64, 128, 256]
        },
        'timestep': {
            'distribution': "int_uniform",
            'min': 1,
            'max': 12
        },
        'lr': {
            'values': [1e-2, 1e-1]
        },
        'layers': {
            'values': [1, 2, 4, 8]
        },
        
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="ancy", project="NATAL-2022-06-22-SIMPLE")

Create sweep with ID: 9m90flkc
Sweep URL: https://wandb.ai/ancy/NATAL-2022-06-22-SIMPLE/sweeps/9m90flkc


In [ ]:
def create_rnn():
    config_defaults = {
    'batch_size': 128,
    'lr': 1e-3,
    'units': 64,
    'timestep': 3,
    'dropout': 0.3,
    'layers': 1,
    }

    wandb.init(config=config_defaults)
    config = wandb.config

    batch_size = config.batch_size
    lr         = config.lr
    num_units  = config.units
    timestep   = config.timestep
    dropout    = config.dropout
    layers     = config.layers

    train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y = make_data2(hourly, config.timestep, resid_check=False)
    model = Sequential()

    if layers > 1:
      model.add(SimpleRNN(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout, return_sequences=True))

      for i in range(layers-2):
        model.add(SimpleRNN(units = num_units, dropout=dropout, return_sequences=True))

      model.add(SimpleRNN(units = num_units, dropout=dropout))

    else:
      model.add(SimpleRNN(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout))

    model.add(Dense(units = 1))

    model.compile(
        loss="mse",
        optimizer=Adam(learning_rate=lr)
    )

    model.fit(train_X, train_y, batch_size=batch_size,
              epochs=200, verbose=0, shuffle=False,
              validation_data=(valid_X, valid_y),
              callbacks=[WandbCallback(),
                          EarlyStopping(patience=10, restore_best_weights=True)])
    
    yhat = model.predict(test_X)
    yhat_inv = scaler_y.inverse_transform(yhat)
    #resid_sum = (yhat_inv+sarima_pred.values[config.timestep:])
    #resid_sum2 = (yhat_inv+sarima_pred2.values[config.timestep:])

    wandb.log({"Test_MSE": mean_squared_error(yhat_inv, scaler_y.inverse_transform(test_y))})

    wandb.log({"Test_RMSE": sqrt(mean_squared_error(yhat_inv, scaler_y.inverse_transform(test_y)))})

    wandb.log({"Test_MAE": mean_absolute_error(yhat_inv, scaler_y.inverse_transform(test_y))})

    wandb.log({"Test_MAPE": mean_absolute_percentage_error(yhat_inv, scaler_y.inverse_transform(test_y))})

    return model

In [ ]:
wandb.agent(sweep_id, create_rnn, count=50)

wandb: Agent Starting Run: f030dpe5 with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.3
wandb: 	layers: 8
wandb: 	lr: 0.1
wandb: 	timestep: 9
wandb: 	units: 256
wandb: Currently logged in as: ancy. Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▁▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇██
loss,▇█▅▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▂█▅▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_MAE,1.27131
Test_MAPE,0.20883
Test_MSE,2.5323
Test_RMSE,1.59132


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 2fn7ode3 with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0
wandb: 	layers: 4
wandb: 	lr: 0.01
wandb: 	timestep: 12
wandb: 	units: 256


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▂▂▃▃▄▅▅▆▆▇▇█
loss,█▂▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁
Test_MAE,1.26514
Test_MAPE,0.19975
Test_MSE,2.78539
Test_RMSE,1.66895


...

Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_MAE,1.16183
Test_MAPE,0.18572
Test_MSE,2.17528
Test_RMSE,1.47488


wandb: Agent Starting Run: 9842f2dy with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.3
wandb: 	layers: 4
wandb: 	lr: 0.01
wandb: 	timestep: 1
wandb: 	units: 128


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▅▅▆▂▄▂▁▁▁▁▁▂▁▁▁▁▁▁
Test_MAE,0.86719
Test_MAPE,0.1488
Test_MSE,1.22678
Test_RMSE,1.1076


wandb: Agent Starting Run: vlnbpozo with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.3
wandb: 	layers: 2
wandb: 	lr: 0.01
wandb: 	timestep: 5
wandb: 	units: 128


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_MAE,0.85572
Test_MAPE,0.14907
Test_MSE,1.3301
Test_RMSE,1.1533


# LSTM

In [ ]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'name': 'LSTM',
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'best_val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'batch_size': {
            'values': [256, 128, 64, 32]
        },
        'dropout': {
            'values': [0, 0.1, 0.3]
        },
        'units': {
            'values': [64, 128, 256]
        },
        'timestep': {
            'distribution': "int_uniform",
            'min': 1,
            'max': 12
        },
        'lr': {
            'values': [1e-2, 1e-1]
        },
        'layers': {
            'values': [1, 2, 4, 8]
        },
        
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="ancy", project="NATAL-2022-06-22-SIMPLE")

Create sweep with ID: tq02dgks
Sweep URL: https://wandb.ai/ancy/NATAL-2022-06-22-SIMPLE/sweeps/tq02dgks


In [ ]:
def create_lstm():
    config_defaults = {
    'batch_size': 128,
    'lr': 1e-3,
    'units': 64,
    'timestep': 3,
    'dropout': 0.3,
    'layers': 1,
    }

    wandb.init(config=config_defaults)
    config = wandb.config

    batch_size = config.batch_size
    lr         = config.lr
    num_units  = config.units
    timestep   = config.timestep
    dropout    = config.dropout
    layers     = config.layers

    train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y = make_data2(hourly, config.timestep, resid_check=False)
    model = Sequential()

    if layers > 1:
      model.add(LSTM(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout, return_sequences=True))

      for i in range(layers-2):
        model.add(LSTM(units = num_units, dropout=dropout, return_sequences=True))

      model.add(LSTM(units = num_units, dropout=dropout))

    else:
      model.add(LSTM(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout))

    model.add(Dense(units = 1))
    model.compile(
        loss="mse",
        optimizer=Adam(learning_rate=lr)
    )

    model.fit(train_X, train_y, batch_size=batch_size,
              epochs=200, verbose=0, shuffle=False,
              validation_data=(valid_X, valid_y),
              callbacks=[WandbCallback(),
                          EarlyStopping(patience=10, restore_best_weights=True)])
    
    yhat = model.predict(test_X)
    yhat_inv = scaler_y.inverse_transform(yhat)
    #resid_sum = (yhat_inv+sarima_pred.values[config.timestep:])
    #resid_sum2 = (yhat_inv+sarima_pred2.values[config.timestep:])

    wandb.log({"Test_MSE": mean_squared_error(yhat_inv, scaler_y.inverse_transform(test_y))})

    wandb.log({"Test_RMSE": sqrt(mean_squared_error(yhat_inv, scaler_y.inverse_transform(test_y)))})

    wandb.log({"Test_MAE": mean_absolute_error(yhat_inv, scaler_y.inverse_transform(test_y))})

    wandb.log({"Test_MAPE": mean_absolute_percentage_error(yhat_inv, scaler_y.inverse_transform(test_y))})

    return model

In [ ]:
wandb.agent(sweep_id, create_lstm, count=50)

wandb: Agent Starting Run: vgtfjso3 with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0
wandb: 	layers: 8
wandb: 	lr: 0.01
wandb: 	timestep: 12
wandb: 	units: 256


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_MAE,1.15859
Test_MAPE,0.19015
Test_MSE,2.21611
Test_RMSE,1.48866


wandb: Agent Starting Run: 7q873w15 with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0
wandb: 	layers: 2
wandb: 	lr: 0.1
wandb: 	timestep: 5
wandb: 	units: 128


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▂▂▃▃▄▄▅▅▆▆▇▇█
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▅█▅▁▇▆▅▂▅▃▂▃▃▄
Test_MAE,1.17217
Test_MAPE,0.20479
Test_MSE,2.1578
Test_RMSE,1.46895


wandb: Agent Starting Run: ifnzz8yy with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.3
wandb: 	layers: 1
wandb: 	lr: 0.1
wandb: 	timestep: 12
wandb: 	units: 64


...

Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▅█▄▂▂▃▃▃▂▂▃▂▂▂▁▂▂▁▁▂▂▁▁▂▁▁▂▁▁▁▁▁▂▁▂▁▁▁
Test_MAE,0.80995
Test_MAPE,0.143
Test_MSE,1.09786
Test_RMSE,1.04779


wandb: Agent Starting Run: cjr6zdmp with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.3
wandb: 	layers: 1
wandb: 	lr: 0.1
wandb: 	timestep: 12
wandb: 	units: 64


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▆▅▄▇▅▃▃▁▂▆▃▁█▄▂▃▂▂▅▂▄▃
Test_MAE,0.97382
Test_MAPE,0.20766
Test_MSE,4.60958
Test_RMSE,2.14699


wandb: Agent Starting Run: uqsq9230 with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.3
wandb: 	layers: 1
wandb: 	lr: 0.1
wandb: 	timestep: 10
wandb: 	units: 128


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▃▃▃▃▄█▄▂▂▂▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁
Test_MAE,0.8204
Test_MAPE,0.16822
Test_MSE,1.62661
Test_RMSE,1.27538


# GRU

In [ ]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'name': 'GRU',
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'best_val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'batch_size': {
            'values': [256, 128, 64, 32]
        },
        'dropout': {
            'values': [0, 0.1, 0.3]
        },
        'units': {
            'values': [64, 128, 256]
        },
        'timestep': {
            'distribution': "int_uniform",
            'min': 1,
            'max': 12
        },
        'lr': {
            'values': [1e-2, 1e-1]
        },
        'layers': {
            'values': [1, 2, 4, 8]
        },
        
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="ancy", project="NATAL-2022-06-22-SIMPLE")

Create sweep with ID: uu8j2e56
Sweep URL: https://wandb.ai/ancy/NATAL-2022-06-22-SIMPLE/sweeps/uu8j2e56


In [ ]:
def create_gru():
    config_defaults = {
    'batch_size': 128,
    'lr': 1e-3,
    'units': 64,
    'timestep': 3,
    'dropout': 0.3,
    'layers': 1,
    }

    wandb.init(config=config_defaults)
    config = wandb.config

    batch_size = config.batch_size
    lr         = config.lr
    num_units  = config.units
    timestep   = config.timestep
    dropout    = config.dropout
    layers     = config.layers

    train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y = make_data2(hourly, config.timestep, resid_check=False)
    model = Sequential()

    if layers > 1:
      model.add(GRU(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout, return_sequences=True))

      for i in range(layers-2):
        model.add(GRU(units = num_units, dropout=dropout, return_sequences=True))

      model.add(GRU(units = num_units, dropout=dropout))

    else:
      model.add(GRU(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout))

    model.add(Dense(units = 1))
    lr = config.lr
    model.compile(
        loss="mse",
        optimizer=Adam(learning_rate=lr)
    )
    
    model.fit(train_X, train_y, batch_size=config.batch_size,
              epochs=200, verbose=0, shuffle=False,
              validation_data=(valid_X, valid_y),
              callbacks=[WandbCallback(),
                          EarlyStopping(patience=10, restore_best_weights=True)])
    
    yhat = model.predict(test_X)
    yhat_inv = scaler_y.inverse_transform(yhat)
    #resid_sum = (yhat_inv+sarima_pred.values[config.timestep:])
    #resid_sum2 = (yhat_inv+sarima_pred2.values[config.timestep:])

    wandb.log({"Test_MSE": mean_squared_error(yhat_inv, scaler_y.inverse_transform(test_y))})

    wandb.log({"Test_RMSE": sqrt(mean_squared_error(yhat_inv, scaler_y.inverse_transform(test_y)))})

    wandb.log({"Test_MAE": mean_absolute_error(yhat_inv, scaler_y.inverse_transform(test_y))})

    wandb.log({"Test_MAPE": mean_absolute_percentage_error(yhat_inv, scaler_y.inverse_transform(test_y))})

    return model

In [ ]:
wandb.agent(sweep_id, create_gru, count=50)

wandb: Agent Starting Run: syum4pgr with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.1
wandb: 	layers: 1
wandb: 	lr: 0.01
wandb: 	timestep: 11
wandb: 	units: 256


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
loss,█▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▂█▃▆▂▄▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_MAE,0.74195
Test_MAPE,0.12805
Test_MSE,1.07908
Test_RMSE,1.03879


wandb: Agent Starting Run: 9qcid0nj with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.3
wandb: 	layers: 8
wandb: 	lr: 0.01
wandb: 	timestep: 3
wandb: 	units: 64


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▇█▆▆▅▄▂▂▁▁▂▁▂▂▂▂▃▄▄▃
Test_MAE,0.88199
Test_MAPE,0.15671
Test_MSE,1.42262
Test_RMSE,1.19274


wandb: Agent Starting Run: affb9k16 with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0
wandb: 	layers: 2
wandb: 	lr: 0.01
wandb: 	timestep: 2
wandb: 	units: 64


...

Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▆█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_MAE,0.78273
Test_MAPE,0.13033
Test_MSE,1.4636
Test_RMSE,1.2098


wandb: Agent Starting Run: rc2drolq with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0
wandb: 	layers: 2
wandb: 	lr: 0.01
wandb: 	timestep: 9
wandb: 	units: 256


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
loss,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_MAE,0.85802
Test_MAPE,0.27305
Test_MSE,2.83377
Test_RMSE,1.68338


wandb: Agent Starting Run: kugnp4w2 with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.1
wandb: 	layers: 2
wandb: 	lr: 0.01
wandb: 	timestep: 6
wandb: 	units: 128


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_MAE,0.72266
Test_MAPE,0.12668
Test_MSE,1.01124
Test_RMSE,1.00561


# Transformer

In [ ]:
sweep_config = {
    'name': 'New_Transformer',
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'best_val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'batch_size': {
            'values': [128, 64, 32]
        },
        'dropout': {
            'values': [0, 0.1, 0.3]
        },
        'timestep': {
            'distribution': "int_uniform",
            'min': 1,
            'max': 12
        },
        'n_heads': {
            'values': [2, 4, 8]
        },
        'ff_dim': {
            'values': [128,256]
        },
        'n_blocks': {
            'values': [2, 4, 8]
        },
        'embed_dim': {
            'values': [32, 64, 128]
        },
        'time_2_vec': {
            'values': [2,3,4]
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="ancy", project="NATAL-2022-06-22-SIMPLE")

Create sweep with ID: 4tftva6n
Sweep URL: https://wandb.ai/ancy/NATAL-2022-06-22-SIMPLE/sweeps/4tftva6n


In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, feat_dim, num_heads, ff_dim, rate = 0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads = num_heads, key_dim = embed_dim)
        self.ffn = keras.Sequential( [layers.Dense(ff_dim, activation = "gelu"), layers.Dense(feat_dim),] )
        self.layernorm1 = layers.BatchNormalization()
        self.layernorm2 = layers.BatchNormalization()
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.embed_dim = embed_dim
        self.feat_dim = feat_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training = training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training = training)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):

        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'feat_dim': self.feat_dim,
            'num_heads': self.num_heads,
            'ff_dim': self.ff_dim,
            'rate': self.rate,
        })
        return config

In [ ]:
class Time2Vec(keras.layers.Layer):
    def __init__(self, kernel_size = 1):
        super(Time2Vec, self).__init__(trainable = True, name = 'Time2VecLayer')
        self.k = kernel_size

    def build(self, input_shape):
        # trend
        self.wb = self.add_weight(name = 'wb', shape = (input_shape[1],), initializer = 'uniform', trainable = True)
        self.bb = self.add_weight(name = 'bb', shape = (input_shape[1],), initializer = 'uniform', trainable = True)
        # periodic
        self.wa = self.add_weight(name = 'wa', shape = (1, input_shape[1], self.k), initializer = 'uniform', trainable = True)
        self.ba = self.add_weight(name = 'ba', shape = (1, input_shape[1], self.k), initializer = 'uniform', trainable = True)
        super(Time2Vec, self).build(input_shape)

    def call(self, inputs, **kwargs):
        bias = self.wb * inputs + self.bb
        dp = K.dot(inputs, self.wa) + self.ba
        wgts = K.sin(dp) # or K.cos(.)
        ret = K.concatenate([K.expand_dims(bias, -1), wgts], -1)
        ret = K.reshape(ret, (-1, inputs.shape[1] * (self.k + 1)))
        return ret

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1] * (self.k + 1))

    def get_config(self):

        config = super().get_config()
        config.update({
            'kernel_size': self.k,
        })
        return config

In [ ]:
EPOCHS = 50
N_HEADS = 8
N_FOLDS = 10
FF_DIM = 256
N_BLOCKS = 6
EMBED_DIM = 64
BATCH_SIZE = 16
WINDOW_SIZE = 65
DROPUT_RATE = 0.0
TIME_2_VEC_DIM = 3
TRAIN_MODEL = True
SKIP_CONNECTION_STRENGTH = 0.9

In [ ]:
def get_model(input_shape, time2vec_dim = 3):
    inp = Input(input_shape)
    x = inp

    time_embedding = keras.layers.TimeDistributed(Time2Vec(time2vec_dim - 1))(x)
    x = Concatenate(axis = -1)([x, time_embedding])
    x = layers.LayerNormalization(epsilon = 1e-6)(x)

    for k in range(N_BLOCKS):
        x_old = x
        transformer_block = TransformerBlock(EMBED_DIM, input_shape[-1] + ( input_shape[-1] * time2vec_dim), N_HEADS, FF_DIM, DROPUT_RATE)
        x = transformer_block(x)
        x = ((1.0 - SKIP_CONNECTION_STRENGTH) * x) + (SKIP_CONNECTION_STRENGTH * x_old)

    x = layers.Flatten()(x)

    x = layers.Dense(128, activation = "selu")(x)
    x = layers.Dropout(DROPUT_RATE)(x)
    x = Dense(1, activation = 'linear')(x)

    out = x
    model = Model(inp, out)
    return model

In [ ]:
def get_model2():

    config_defaults = {
    'batch_size': 128,
    'lr': 1e-3,
    'timestep': 3,
    'n_heads': 8,
    'ff_dim':256,
    'n_blocks':6,
    'embed_dim':64,
    'dropout':0,
    'time_2_vec':3,
    }

    wandb.init(config=config_defaults)
    config = wandb.config
    train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y = make_data2(hourly, config.timestep, resid_check=False)


    batch_size=config.batch_size
    lr=config.lr
    N_HEADS = config.n_heads
    FF_DIM = config.ff_dim
    N_BLOCKS = config.n_blocks
    EMBED_DIM = config.embed_dim
    DROPUT_RATE = config.dropout
    time2vec_dim = config.time_2_vec

    input_shape = train_X.shape[1:]
    inp = Input(input_shape)
    x = inp

    time_embedding = keras.layers.TimeDistributed(Time2Vec(time2vec_dim - 1))(x)
    x = Concatenate(axis = -1)([x, time_embedding])
    x = layers.LayerNormalization(epsilon = 1e-6)(x)

    for k in range(N_BLOCKS):
        x_old = x
        transformer_block = TransformerBlock(EMBED_DIM, input_shape[-1] + ( input_shape[-1] * time2vec_dim), N_HEADS, FF_DIM, DROPUT_RATE)
        x = transformer_block(x)
        x = ((1.0 - SKIP_CONNECTION_STRENGTH) * x) + (SKIP_CONNECTION_STRENGTH * x_old)

    x = layers.Flatten()(x)

    x = layers.Dense(128, activation = "selu")(x)
    x = layers.Dropout(DROPUT_RATE)(x)
    x = Dense(1, activation = 'linear')(x)

    out = x
    model = Model(inp, out)

    model.compile(
        loss="mse",
        optimizer=Adam(learning_rate=lr)
                  )

    model.fit(train_X, train_y, batch_size=config.batch_size,
              epochs=200, verbose=0, shuffle=False,
              validation_data=(valid_X, valid_y),
              callbacks=[WandbCallback(),
                          EarlyStopping(patience=10, restore_best_weights=True)]
              )
    
    yhat = model.predict(test_X)
    yhat_inv = scaler_y.inverse_transform(yhat)
    #resid_sum = (yhat_inv+sarima_pred.values[config.timestep:])
    #resid_sum2 = (yhat_inv+sarima_pred2.values[config.timestep:])

    wandb.log({"Test_MSE": mean_squared_error(yhat_inv, scaler_y.inverse_transform(test_y))})

    wandb.log({"Test_RMSE": sqrt(mean_squared_error(yhat_inv, scaler_y.inverse_transform(test_y)))})

    wandb.log({"Test_MAE": mean_absolute_error(yhat_inv, scaler_y.inverse_transform(test_y))})

    wandb.log({"Test_MAPE": mean_absolute_percentage_error(yhat_inv, scaler_y.inverse_transform(test_y))})
    
    return model

In [ ]:
wandb.agent(sweep_id, get_model2, count=50)

wandb: Agent Starting Run: 15mtewwh with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.3
wandb: 	embed_dim: 128
wandb: 	ff_dim: 256
wandb: 	n_blocks: 2
wandb: 	n_heads: 8
wandb: 	time_2_vec: 3
wandb: 	timestep: 9


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▁▂▂▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_MAE,0.83854
Test_MAPE,0.14762
Test_MSE,1.16316
Test_RMSE,1.0785


wandb: Agent Starting Run: nww7rjbe with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.1
wandb: 	embed_dim: 64
wandb: 	ff_dim: 256
wandb: 	n_blocks: 4
wandb: 	n_heads: 4
wandb: 	time_2_vec: 4
wandb: 	timestep: 9


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▅▃▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▅▂█▂▅▂▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_MAE,0.9415
Test_MAPE,0.21233
Test_MSE,1.64567
Test_RMSE,1.28284


wandb: Agent Starting Run: xjj7phtz with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.3
wandb: 	embed_dim: 64
wandb: 	ff_dim: 128
wandb: 	n_blocks: 4
wandb: 	n_heads: 8
wandb: 	time_2_vec: 3
wandb: 	timestep: 8


...

Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
loss,█▃▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁█▃▂▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_MAE,0.9164
Test_MAPE,0.15089
Test_MSE,1.37751
Test_RMSE,1.17367


wandb: Agent Starting Run: xrs2dyzo with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.3
wandb: 	embed_dim: 128
wandb: 	ff_dim: 256
wandb: 	n_blocks: 4
wandb: 	n_heads: 8
wandb: 	time_2_vec: 4
wandb: 	timestep: 2


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
loss,█▅▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▇█▂▃▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_MAE,0.8007
Test_MAPE,0.14021
Test_MSE,1.08344
Test_RMSE,1.04089


wandb: Agent Starting Run: 39gc70pa with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.3
wandb: 	embed_dim: 64
wandb: 	ff_dim: 256
wandb: 	n_blocks: 4
wandb: 	n_heads: 8
wandb: 	time_2_vec: 3
wandb: 	timestep: 4


Test_MAE,▁
Test_MAPE,▁
Test_MSE,▁
Test_RMSE,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▅▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▃▃▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test_MAE,0.78958
Test_MAPE,0.16519
Test_MSE,1.05877
Test_RMSE,1.02897
